In [7]:
## tf2021callerTCP
 
import socket
import time

port = 8005
host = 'localhost' ##'192.168.1.62'
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

acceptinrequest  = "msg(acceptIn,request, python,acceptinmanager,acceptIn(ID1),  1)"
entercarrequest  = "msg(carenter,request, python,carentermanager,carenter(S,N), 2)"
##entercarrequest  = "msg(carentermanager,request, python,carentermanager,carenter(S,N), 2)"

cmddispatch   = "msg(cmd, dispatch,python,basicrobot,cmd(VALUE),1)"
enddispatch   = "msg(cmd, dispatch,python,basicrobot,end(VALUE),1)"
 

def connect(port) :
    server_address = (host, port)      ##localhost
    sock.connect(server_address)    
    print("CONNECTED " , server_address)

def emit( event ) :
    print("emit ", event)
    msg = event + "\n"
    byt=msg.encode()    #required in Python3
    sock.send(byt)
    
def forward( message ) :
    print("forward ", message)
    msg = message + "\n"
    byt=msg.encode()    #required in Python3
    sock.send(byt)
    ##receiveALine()

def request( message ) :
    print("request ", message)
    msg = message + "\n"
    byt=msg.encode()    #required in Python3
    sock.send(byt)
    if msg.startswith("msg(acceptIn") :
        handleAnswerAcceptIn()
    if msg.startswith("msg(carenter") :
        handleAnswerEnter()

def receiveALine() :    
    print("receiveALine " )
    reply = ''
    while True:
        answer = sock.recv(50)
        print("answer len=", len(answer))
        if len(answer) <= 0 :
            break
        reply += answer.decode("utf-8")
        ## print("reply=", reply)
        if reply.endswith("\n") :
            break
    print("final reply=", reply)


def handleAnswer() :
    while True:  ##client wants to maintain the connection
        reply = ''
        while True:
            answer = sock.recv(50)
            ## print("answer len=", len(answer))
            if len(answer) <= 0 :
                break
            reply += answer.decode("utf-8")
            ## print("reply=", reply)
            if reply.endswith("\n") :
                break
        print("final reply=", reply)
        break
    #reply=msg(enter,reply,acceptinmanager,python,enter(1,9901,5000),47)
    return reply

    
def handleAnswerAcceptIn() :
    print("handleAnswerAcceptIn " )
    reply   = handleAnswer()
    penter  = reply.find("enter(")
    msg     = reply[penter+6:] ##  enter(1,9901,5000),47)
    ## print("handleAnswerAcceptIn msg=", msg)
    splitted= msg.split(",") 
    SLOTNUM = splitted[0]
    NONCE   = splitted[1]
    req     = entercarrequest.replace("S",SLOTNUM).replace("N",NONCE)  
    ## print("handleAnswerAcceptIn req=", req)
    request( req )        

def handleAnswerEnter() :
    print("handleAnswerEnter " )
    reply     = handleAnswer()      
    preceipt  = reply.find("receipt(") 
    msg       = reply[preceipt+8:] ##  receipt(T57F1),88)
    ## print("handleAnswerEnter msg=", msg)
    splitted  = msg.split(",") 
    tokenid   = splitted[0].replace(")","")
    print("tokenid=", tokenid)
    
def terminate() :
    sock.close()
    print("BYE")
    
def console() :  
    v =  str( input() )
    print("INPUT" , v  )    
    while( v != "q"  ) :
        request( acceptinrequest )
        ## print( v )        
        v = str(input() )      

###########################################    
connect(port)
console()
##terminate()  


CONNECTED  ('localhost', 8005)
a
INPUT a
request  msg(acceptIn,request, python,acceptinmanager,acceptIn(ID1),  1)
handleAnswerAcceptIn 
final reply= msg(enter,reply,acceptinmanager,python,enter(2,3927,5000),151)

request  msg(carenter,request, python,carentermanager,carenter(2,3927), 2)
handleAnswerEnter 
final reply= msg(receipt,reply,carentermanager,python,receipt(TFSR2),194)

tokenid= TFSR2
q
